In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop

from model_2d.NMR_gcn import NMR_GCN

from train_evaluate_2d import NMR_prediction

from preprocess.create_node_embedding.node_embeddings_glycoscience import create_node_embeddings

from preprocess.build_adjaency_matrix.create_adjaency_matrix_glycoscience import build_adjacency_matrix

from preprocess.create_graph.create_graph_data_glycosciencedb import create_graph

# from dgl import save_graphs, load_graphs
# from tqdm import tqdm
from dgl import save_graphs, load_graphs

##### A Baseline model for Glycosciencedb using random forest

We follow the same way of loading and testing the data as 2D GNN model, please refer 2D_GODESS.ipynb.

Here, we encode the atom-level and monosaccharide-level features (fingerprints) as node embeddings.

However, we use atom features as input to the random forest model, but ignore the atom connections.

In [2]:
atom_dim = 32

residual_dim = 16

mono_dim = 8
ab_dim = 8
dl_dim = 8
pf_dim = 8

seed=9721

In [3]:
C = create_node_embeddings(seed=seed)

df_atom_embedding, df_residual_embedding, df_monosaccharide_embedding, df_ab_embedding, df_dl_embedding, df_pf_embedding =\
C.create_all_embeddings(atom_dim=atom_dim, residual_dim=residual_dim, mono_dim=mono_dim, 
                       ab_dim=ab_dim, dl_dim=dl_dim, pf_dim=pf_dim)



100%|████████████████████████████████████████| 299/299 [00:00<00:00, 700.50it/s]


In [4]:
df_atom_embedding.to_csv(C.out_atom_embed, index=False)
df_residual_embedding.to_csv(C.out_residual_embed, index=False)
df_monosaccharide_embedding.to_csv(C.out_monosaccharide_embed, index=False)

df_ab_embedding.to_csv(C.out_bound_ab, index = False) 
df_dl_embedding.to_csv(C.out_bound_dl, index = False) 
df_pf_embedding.to_csv(C.out_carbon_pf, index = False) 

In [5]:
B = build_adjacency_matrix()
B.calculate_all_matrix()

100%|█████████████████████████████████████████| 299/299 [00:04<00:00, 69.85it/s]


In [6]:
num_test = 60 # 299 * 20% 
Create = create_graph(num_test=num_test, seed=seed)
g, test_index = Create.create_all_graph()

--------------------------loading NMR Graph-------------------------------


100%|█████████████████████████████████████████| 299/299 [00:03<00:00, 99.46it/s]


In [13]:
features = g.ndata["feat"]
labels = g.ndata["shift_value"]

masks = g.ndata['train_hydrogen_mask'], g.ndata['test_hydrogen_mask']
# masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']

In [14]:
train_x = features.numpy()[masks[0]]
test_x = features.numpy()[masks[1]]

train_y = labels.numpy()[masks[0]]
test_y = labels.numpy()[masks[1]]

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regr = RandomForestRegressor(random_state = 0)

regr.fit(train_x, train_y)

RandomForestRegressor(random_state=0)

In [16]:
pred_y = regr.predict(test_x)

In [17]:
import numpy as np

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [18]:
rmse(pred_y, test_y)

0.1776719184286958